In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict, Counter
import glob
import json
import pickle as pkl
import ast
import os
from sklearn.metrics.pairwise import cosine_similarity
from random import sample
from transformers import BertTokenizer, BertModel
import pandas as pd
import numpy as np
from collections import defaultdict, Counter
import glob
import json
import pickle as pkl
import ast
import os
from random import sample
import networkx as nx

from random import sample
import sklearn
import re
import string
import warnings
from bs4 import BeautifulSoup
import gensim

from pandarallel import pandarallel
pandarallel.initialize()

INFO: Pandarallel will run on 128 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')
model = BertModel.from_pretrained("bert-base-multilingual-uncased")

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
# LEVEL1_FOLDERS = ['groups']
allfiles = []

# for l in LEVEL1_FOLDERS:
#     l1path = '/data/shruti/ONR/big_data/facebook/' + l + '/*'
#     LEVEL2_FOLDERS = glob.glob(l1path)
#     for l2 in LEVEL2_FOLDERS:
#         l2path = l2 + "/*.json"
#         fnames = glob.glob(l2path)
#         for f in fnames:
#             if os.stat(f).st_size == 0:
#                 fakevar=1
#             else:
#                 allfiles.append(f)

allfiles = ['/data/shruti/ONR/big_data/facebook/groups/re_hindi/2021-07-01_2021-08-01.json']


In [4]:
def get_contents(fileName):
    dictlist = []
    with open(fileName, "r") as jfile:
        for line in jfile:
            tempdict = {}
            try:
                job = json.loads(line)
            except:
                job = None
            if job!=None:    
                if "date" in job:
                    tempdict['date'] = job['date']
                    if "title" in job:
                        tempdict['title'] = job['title']
                    if "caption" in job:
                        tempdict['caption'] = job['caption']
                    if "description" in job:
                        tempdict['description'] = job['description']
                    if "description" in job:
                        tempdict['description'] = job['description']
                    if "message" in job:
                        tempdict['message'] = job['message']
                    if "link" in job:
                        tempdict['link'] = job['link']
                    if "postUrl" in job:
                        tempdict['postUrl'] = job['postUrl']
                    if "score" in job:
                        tempdict['score'] = job['score']

                    if "account" in job:
                        try:
                            tempdict['handle'] = job['account']['name']
                        except:
                            fakevar=1

                        try:
                            tempdict['platformId'] = job['account']['platformId']
                        except:
                            fakevar=1

                        try:
                            tempdict['id'] = job['account']['id']
                        except:
                            fakevar=1

                    dictlist.append(tempdict)

    if len(dictlist) > 0:
        return dictlist


In [5]:
filcontent = pd.DataFrame()
filcontent['fileNames'] = allfiles
filcontent['content'] = filcontent['fileNames'].parallel_apply(lambda x: get_contents(x))

In [6]:
jsonList = []

for idx, row in filcontent.iterrows():
    for c in row['content']:
        jsonList.append(c)

In [7]:
fdata = pd.DataFrame(jsonList)

In [8]:
print(len(fdata))
fdata = fdata.dropna(subset=['message'])
print(len(fdata))

567297
293716


In [9]:
fdata = fdata.drop_duplicates()
len(fdata)

293716

In [10]:
fdata = fdata.drop_duplicates(subset=['handle','message'])
len(fdata)

263520

In [11]:
import re
import string
import warnings
from bs4 import BeautifulSoup

emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U0001F1F2-\U0001F1F4"  # Macau flag
        u"\U0001F1E6-\U0001F1FF"  # flags
        u"\U0001F600-\U0001F64F"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U0001F1F2"
        u"\U0001F1F4"
        u"\U0001F620"
        u"\u200d"
        u"\u2640-\u2642"
        "]+", flags=re.UNICODE)




remove_urls = lambda x: re.sub("http(.+)?(\W|$)", ' ', x)
remove_RT = lambda x: x.replace("RT ", "")
remove_mentions = lambda x: re.sub("@\S+", '', x)
remove_hashtags = lambda x: re.sub("#\S+", '', x)
remove_digits = lambda x: re.sub("\d+", "", x)
remove_punct = lambda x: re.sub("!|\||\%|\.|\-|\/|:|…|,|\?|।+|'|⁉|\*|‘|’|\"|\(|\)+", "", x)
remove_emojis = lambda x: emoji_pattern.sub("", x)
normalize_spaces = lambda x: re.sub("[\n\r\t ]+", ' ', x)


In [12]:
remove_noise = lambda x:normalize_spaces(
                                remove_emojis(
                                    remove_punct(
                                        remove_digits(
                                            remove_hashtags(
                                                remove_mentions(
                                                    remove_punct(
                                                        remove_urls(x.lower()))))))))


In [13]:
fdata['clean_text'] = fdata['message'].parallel_apply(lambda x: remove_noise(x) if len(x) >15 else '0')

In [14]:
frame = fdata.copy()

In [15]:
print(len(frame))
frame = frame.loc[frame['clean_text']!='0']
print(len(frame))

263520
255479


In [16]:
frame['textlen'] = frame['clean_text'].parallel_apply(lambda x: len(x.split(" ")))

In [17]:
frame = frame.loc[(frame['textlen']>14)&(frame['textlen']<240)]
len(frame)

137899

In [18]:
frame = frame.loc[frame['title']!='This is a re-share of a post']
frame = frame.drop_duplicates(subset=['handle','message'])
print(len(frame))

136875


In [19]:
sample_frame = frame#.sample(100000)
sample_text = sample_frame['clean_text'].tolist()

In [20]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('distiluse-base-multilingual-cased-v2')

embeddings = model.encode(sample_text)

In [21]:
embeddings.shape

(136875, 512)

In [22]:
month = 'july_'
sframefile = "/data/shruti/ONR/small_data/embeddings/" + month + "frame.csv"
sembedfile = "/data/shruti/ONR/small_data/embeddings/" + month + "embeddings.pkl"

In [23]:
sframefile

'/data/shruti/ONR/small_data/embeddings/july_frame.csv'

In [24]:
sample_frame.to_csv(sframefile)

In [25]:
import pickle as pkl

with open(sembedfile, "wb") as pfile:
    pkl.dump(embeddings, pfile)